Import Libraries

In [2]:
import pandas as pd

/var/folders/lp/hgbg7xdj7ql_zq41nk0lt6yw0000gn/T/ipykernel_48041/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Holding Data

In [7]:
user_cols = ['id', 'age', 'sex', 'occupation', 'zip_code']
user_pd = pd.read_csv('../Dataset/ml-100k/u.user', sep='|', names=user_cols, encoding='latin-1')
user_pd.shape

(943, 5)

In [10]:
rate_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

rating_base = pd.read_csv('../Dataset/ml-100k/ua.base', sep='\t', names=rate_cols, encoding='latin-1')
rating_test = pd.read_csv('../Dataset/ml-100k/ua.test', sep='\t', names=rate_cols, encoding='latin-1')

rating_base.shape, rating_test.shape

((90570, 4), (9430, 4))